In [2]:
from os import listdir
import pandas as pd
from tqdm import tqdm
import re
import os

In [4]:
# file_path
hanzi_table = "../一二级汉字表.txt"
pinyin_table = "../拼音汉字表.txt"
yuliao_data = "../sentences_normal.txt" # 预处理后的语料库路径
pinyin_data = "../pinyin_normal.txt"
process0_path = "./0_Process/"
data0_path = "./0_data/"
process1_path = "./1_Process/"
data1_path = "./1_data/"


In [5]:
hanzi_map = {}
duyin_map = {}

with open(hanzi_table, 'r', encoding='utf-8') as f_read1:
    data1 = f_read1.read()
    for i in range(len(data1)):
        hanzi_map[i] = data1[i]
    
with open(pinyin_table, "r", encoding="utf-8") as f_read2:
    lines = f_read2.readlines()
    for i in range(len(lines)):
        line = lines[i].split()
        duyin_map[i] = line[0]

0_Process: 计算transition matrix

In [15]:
# 根据模板文件，多进程操作
with open(process0_path + "00_template.txt") as f:
    template = f.read()
for i in range(46):
    with open(process0_path + f"0_Process{i}.txt", "w") as f:
        rank = i
        file = re.sub("X_NUM_X", str(i), template)
        f.write(file)
    
with open(process0_path + "0_run.txt", "w") as ff:
    for i in range(46):
        ff.write("nohup python " + process0_path+ f"0_Process{i}.txt >> " + process0_path + f"log/P{i}.log 2>&1 &" + '\n')

os.system("sh " + process0_path + "0_run.txt")

0

查看日志文件，当csv文件全部生成后执行下一步

In [6]:
filelist  = listdir(data0_path)
print(len(filelist))

0


In [ ]:
# 合并各进程结果
while True:
    filelist  = listdir(data0_path)
    if len(filelist) == 46:
        dff = pd.read_csv(data0_path + "Process::0.csv")
        for i in tqdm(range(1, 46), desc="reading"):
            dff2 = pd.read_csv(data0_path+ f"Process::{i}.csv")
            dff = dff + dff2
        head_sum = sum(list(dff.iloc[:, -1]))
        dff.iloc[:, -1] = dff.iloc[:, -1]/head_sum
        for i in tqdm(range(6763), desc="merging"):
            row_sum = sum(dff.iloc[i, :-1])
            dff.iloc[i, :-1] = dff.iloc[i, :-1]/row_sum
        dff.to_csv(data0_path+ "0_all.csv", index=0)
        os.system("rm " + process0_path + "log/*.log") # 删除日志文件
        os.system("rm " + data0_path + "Process::{0..45}.csv") # 删除中间文件
        
        break

1_Process: 计算emit_matrix

In [ ]:
# 根据模板文件，多进程操作
with open(process1_path + "1_template.txt") as f:
    template = f.read()
for i in range(46):
    with open(process1_path + f"1_Process{i}.txt", "w") as f:
        rank = i
        file = re.sub("X_NUM_X", str(i), template)
        f.write(file)
    
with open(process1_path+ "1_run.txt", "w") as ff:
    for i in range(46):
        ff.write("nohup python " + process1_path+ f"1_Process{i}.txt >> " + process1_path + f"log/P{i}.log 2>&1 &" + '\n')

os.system("sh " + process1_path + "1_run.txt")

In [ ]:
# 合并各进程结果
while True:
    filelist  = listdir(data1_path)
    if len(filelist) == 46:
        dff_1 = pd.read_csv(data1_path +"Process_0.csv")
        for i in tqdm(range(1, 46), desc="reading"):
            dff2_1 = pd.read_csv(data1_path + f"Process_{i}.csv")
            dff_1 = dff_1 + dff2_1

        for i in tqdm(range(406), desc="merging"):
            row_sum = sum(dff_1.iloc[i, :])
            dff_1.iloc[i, :] = dff_1.iloc[i, :]/row_sum
        dff_1.to_csv(data1_path + "1_all.csv", index=0)
        os.system("rm " + process1_path + "log/*.log") # 删除日志文件
        os.system("rm " + data1_path + "Process_{0..45}.csv") # 删除中间文件

        break